In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#셀레니움
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import time

In [2]:
nb_codzip = pd.read_excel('넘블 챌린지 개방데이터 코드집.xlsx', sheet_name=0)
nb_codzip_finance = pd.read_excel('넘블 챌린지 개방데이터 코드집.xlsx', sheet_name=1)
nb_layout = pd.read_excel('넘블 챌린지 개방데이터 레이아웃.xlsx')
nb_active_out = pd.read_excel('액티브 중소법인 재무보유.xlsx', sheet_name=0)
nb_active_overview = pd.read_excel('액티브 중소법인 재무보유.xlsx', sheet_name=1)
nb_active_oclog = pd.read_excel('액티브 중소법인 재무보유.xlsx', sheet_name=2)
nb_deactive_overview = pd.read_excel('휴폐업 중소법인 재무보유.xlsx', sheet_name=0)
nb_deactive_oclog = pd.read_excel('휴폐업 중소법인 재무보유.xlsx', sheet_name=1)
nb_finance_data = pd.read_csv('재무데이터.txt', encoding='cp949', sep='\t')
nb_active_ind = pd.read_excel('액티브_중소법인_기업개요_산업분류.xlsx')
nb_deactive_ind = pd.read_excel('휴폐업_중소법인_기업개요_산업분류.xlsx')

In [39]:
nb_active_out = pd.read_excel('액티브_중소법인_기업외감_산업분류.xlsx')

In [41]:
nb_active_out

,Unnamed: 0,BIZ_NO,CMP_PFIX_NM,CMP_NM1,CMP_SFIX_NM,CMP_ENM,BZ_TYP,CMP_SCL,PBCO_GB,PSN_CORP_GB,...,ESTB_DATE,ESTB_GB,LIST_DATE,MN_BIZ_CONT,NATN_NM,HOMEPAGE_URL,CEO_NM,EMP_CNT,ind_01_cate,ind_02_cate
0,0,1138111739,NaN,한일가스산업,(주),"HANIL GAS IND. CO.,LTD.",M,2,2,1,...,19660419.0,1,NaN,LP Gas 충전 및 판매업,NaN,www.hanilgas.com,홍순철외 1명,43.0,도매 및 소매업,소매업; 자동차 제외
1,1,2078132193,(주),엠피아이,NaN,"EMPIALEE CO.,LTD.",M,2,2,1,...,19910806.0,1,NaN,가죽 및 대용가죽제품 제조업,NaN,www.mastercnd.co.kr,이스테판상수외 1명,35.0,제조업,"가죽, 가방 및 신발 제조업"
2,2,1318167565,NaN,도영운수,(주),"DOYOUNG TRANSPORTATION CO.,LTD.",M,2,2,1,...,20011030.0,1,NaN,시내버스업,NaN,transport@naver.com,김명화,149.0,운수 및 창고업,육상 운송 및 파이프라인 운송업
3,3,5048128251,(주),중원산업,NaN,"JOONGWON INDUSTRIAL CO.,LTD.",M,2,2,1,...,19990112.0,1,NaN,자동차 부품 제조,NaN,www.koreajw.com,김명구,75.0,제조업,자동차 및 트레일러 제조업
4,4,1098164260,(주),카타나골프,NaN,"CATANA GOLF CO.,LTD.",M,2,2,1,...,20000531.0,1,NaN,골프용품 전반의 제조 및 수출업,NaN,NaN,김홍득,3.0,도매 및 소매업,도매 및 상품 중개업
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28977,28977,8048101453,NaN,제이앤인베스트먼트,NaN,J&INVESTMENT,M,2,2,1,...,20191206.0,1,NaN,경영컨설팅업,NaN,NaN,정경태,0.0,NaN,NaN
28978,28978,8028701430,NaN,디앤엠개발,(주),"DNM Development Co., Ltd.",M,2,2,1,...,20200402.0,1,NaN,비주거용 건물 임대업,NaN,NaN,송성현,1.0,부동산업,부동산업
28979,28979,4268501421,NaN,마스비 건축연구소,NaN,NaN,M,2,2,1,...,19900108.0,1,NaN,"BIM용역, 엔지니어링컨설팅",NaN,NaN,김태남,81.0,"전문, 과학 및 기술 서비스업","건축 기술, 엔지니어링 및 기타 과학기술 서비스업"
28980,28980,6518502462,(주),건설레미콘의정부공장,NaN,"Construction Remicon Uijeongbu Factory Co., Ltd.",M,2,2,1,...,20220615.0,1,NaN,레미콘,NaN,NaN,이미성,26.0,제조업,비금속 광물제품 제조업


In [42]:
nb_ac_ind2 = nb_active_out[['CMP_NM1', 'ind_02_cate']]
nb_ac_ind2.head()

,CMP_NM1,ind_02_cate
0,한일가스산업,소매업; 자동차 제외
1,엠피아이,"가죽, 가방 및 신발 제조업"
2,도영운수,육상 운송 및 파이프라인 운송업
3,중원산업,자동차 및 트레일러 제조업
4,카타나골프,도매 및 상품 중개업


In [7]:
url = 'https://datalab.naver.com/keyword/trendSearch.naver'
driver = webdriver.Chrome(r'C:\pydata\chromedriver.exe')
driver.get(url)

C:\Users\Admin\AppData\Local\Temp\ipykernel_23716\2950796416.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(r'C:\pydata\chromedriver.exe')


In [11]:
driver.find_element(By.CSS_SELECTOR, '#item_keyword1').send_keys('유한아')

In [13]:
driver.find_element(By.CSS_SELECTOR, '#item_sub_keyword1_1').send_keys('유한아,')


In [20]:
for idx in nb_ac_ind2.index :
    url = 'https://datalab.naver.com/keyword/trendSearch.naver'
    driver.get(url)
    driver.implicitly_wait(0.5)
    time.sleep(0.5)
    company = nb_ac_ind2.loc[idx, 'CMP_NM1']
    ind = nb_ac_ind2.loc[idx, 'ind_02_cate']
    driver.find_element(By.CSS_SELECTOR, '#item_keyword1').send_keys(company)
    driver.find_element(By.CSS_SELECTOR, '#item_sub_keyword1_1').send_keys(f'{company}, {ind}')
    driver.find_element(By.CSS_SELECTOR, '#content > div > div.keyword_trend > div.section_step > div > form > fieldset > a > span').click()
    driver.implicitly_wait(0.5)
    time.sleep(0.5)
    driver.find_element(By.CSS_SELECTOR, '#content > div.section_keyword > div.keyword_trend > div.section_ca_board.w_space > div > div > div > div > div > div.graph_area > div.cont_file_down > a').click()
    
    break

In [25]:
df_temp = pd.read_excel(r'C:\Users\Admin\Downloads\datalab.xlsx', header=None)
df_temp

C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,0,1
0,url,http://datalab.naver.com/keyword/trendResult.n...
1,주제,통검
2,범위,합계
3,기간,일간 : 2021-10-16 ~ 2022-10-16
4,성별,"전체(여성,남성)"
...,...,...
368,2022-10-12,73.68421
369,2022-10-13,49.1228
370,2022-10-14,28.07017
371,2022-10-15,0


In [28]:
df_all = pd.DataFrame()
df_all['date'] = date
df_all

,date
8,2021-10-17
9,2021-10-18
10,2021-10-19
11,2021-10-20
12,2021-10-21
...,...
368,2022-10-12
369,2022-10-13
370,2022-10-14
371,2022-10-15


In [29]:
df_all['신우종합건설'] = df_temp.loc[8:, :][1]
df_all

,date,신우종합건설
8,2021-10-17,8.77192
9,2021-10-18,45.61403
10,2021-10-19,33.33333
11,2021-10-20,71.92982
12,2021-10-21,33.33333
...,...,...
368,2022-10-12,73.68421
369,2022-10-13,49.1228
370,2022-10-14,28.07017
371,2022-10-15,0


In [ ]:
df_temp.loc[8:, :]

In [27]:
date = df_temp.loc[8:, :][0]
date

8      2021-10-17
9      2021-10-18
10     2021-10-19
11     2021-10-20
12     2021-10-21
          ...    
368    2022-10-12
369    2022-10-13
370    2022-10-14
371    2022-10-15
372    2022-10-16
Name: 0, Length: 365, dtype: object

# 이거 맞아?

In [53]:
import os
df_all = pd.DataFrame()
df_all['date'] = date
df_all
cnt = 0
for idx in nb_ac_ind2.index :
    url = 'https://datalab.naver.com/keyword/trendSearch.naver'
    driver.get(url)
    driver.implicitly_wait(0.5)
    time.sleep(0.5)
    company = nb_ac_ind2.loc[idx, 'CMP_NM1']
    ind = nb_ac_ind2.loc[idx, 'ind_02_cate']
    driver.find_element(By.CSS_SELECTOR, '#item_keyword1').send_keys(company)
    driver.find_element(By.CSS_SELECTOR, '#item_sub_keyword1_1').send_keys(f'{company}, {ind}')
    driver.find_element(By.CSS_SELECTOR, '#content > div > div.keyword_trend > div.section_step > div > form > fieldset > a > span').click()
    driver.implicitly_wait(0.5)
    time.sleep(0.5)
    driver.find_element(By.CSS_SELECTOR, '#content > div.section_keyword > div.keyword_trend > div.section_ca_board.w_space > div > div > div > div > div > div.graph_area > div.cont_file_down > a').click()
    file_path = r'C:\Users\Admin\Downloads\datalab.xlsx'
    time.sleep(1)
    df_temp = pd.read_excel(file_path, header=None)
    df_temp
    df_all[company] = df_temp.loc[8:, :][1]
    os.remove('C:/Users/Admin/Downloads/datalab.xlsx')


C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\Admin\AppData\Local\Packages\PythonSoftw

In [ ]:
df_all

In [51]:
import os
os.remove('C:/Users/Admin/Downloads/datalab.xlsx')

In [ ]:
os.exi

In [52]:
import glob
glob.glob('C:/Users/Admin/Downloads/datalab *.xlsx')

['C:/Users/Admin/Downloads\\datalab (1).xlsx',
 'C:/Users/Admin/Downloads\\datalab (10).xlsx',
 'C:/Users/Admin/Downloads\\datalab (100).xlsx',
 'C:/Users/Admin/Downloads\\datalab (11).xlsx',
 'C:/Users/Admin/Downloads\\datalab (12).xlsx',
 'C:/Users/Admin/Downloads\\datalab (13).xlsx',
 'C:/Users/Admin/Downloads\\datalab (14).xlsx',
 'C:/Users/Admin/Downloads\\datalab (15).xlsx',
 'C:/Users/Admin/Downloads\\datalab (16).xlsx',
 'C:/Users/Admin/Downloads\\datalab (17).xlsx',
 'C:/Users/Admin/Downloads\\datalab (18).xlsx',
 'C:/Users/Admin/Downloads\\datalab (19).xlsx',
 'C:/Users/Admin/Downloads\\datalab (2).xlsx',
 'C:/Users/Admin/Downloads\\datalab (20).xlsx',
 'C:/Users/Admin/Downloads\\datalab (21).xlsx',
 'C:/Users/Admin/Downloads\\datalab (22).xlsx',
 'C:/Users/Admin/Downloads\\datalab (23).xlsx',
 'C:/Users/Admin/Downloads\\datalab (24).xlsx',
 'C:/Users/Admin/Downloads\\datalab (25).xlsx',
 'C:/Users/Admin/Downloads\\datalab (26).xlsx',
 'C:/Users/Admin/Downloads\\datalab (27).

In [ ]:
df_all

,date,신우종합건설,한일가스산업,랩솔루션
8,2021-10-17,8.77192,0,34.22131
9,2021-10-18,45.61403,0,59.83606
10,2021-10-19,33.33333,0,51.63934
11,2021-10-20,71.92982,0,59.22131
12,2021-10-21,33.33333,0,54.91803
...,...,...,...,...
368,2022-10-12,73.68421,100.0,61.88524
369,2022-10-13,49.1228,0,68.44262
370,2022-10-14,28.07017,0,60.04098
371,2022-10-15,0,0,37.70491


In [30]:
cnt = 1
file_path = f'C:/Users/Admin/Downloads/datalab ({cnt}).xlsx'
pd.read_excel(file_path, header=None)

C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,0,1
0,url,http://datalab.naver.com/keyword/trendResult.n...
1,주제,통검
2,범위,합계
3,기간,일간 : 2021-10-16 ~ 2022-10-16
4,성별,"전체(여성,남성)"
...,...,...
368,2022-10-12,73.68421
369,2022-10-13,49.1228
370,2022-10-14,28.07017
371,2022-10-15,0
